# Pré-Requisitos

## PROV - https://pypi.org/project/prov/

In [ ]:
!pip install prov

## Sweetviz - https://pypi.org/project/sweetviz/

In [ ]:
!pip install sweetviz

# Bibliotecas

In [10]:
import sweetviz as sv
import pandas as pd
import prov
import bs4

# Reprodutibilidade

## Mostra as versões das bibliotecas usadas

In [ ]:
import sys

print('Versão do Pandas:', pd.__version__)
print('Versão do Beautiful Soup:', bs4.__version__)
print('Versão do PROV:', prov.__version__)
print('Versão do Sweetviz:', sv.__version__)
print('Versão do Python:', sys.version)

## Sistema Operacional

In [ ]:
!cat /etc/os-release

### Versão do kernel

In [ ]:
!cat /proc/version

### Arquitetura (32 ou 64 bits)

In [ ]:
!lscpu

# Provenance

In [ ]:
from prov import model
# Create a new provenance document
doc = model.ProvDocument()
# Declaring namespaces
doc.set_default_namespace('https://github.com/ppgi-ufrj-data-science/FakeNews')
doc.add_namespace('kagg', 'https://www.kaggle.com/')
doc.add_namespace('ppgi', 'http://www.ppgi.ufrj.br/')
# Criando entidades
dataset = doc.entity('kagg:fake-news', {model.PROV_TYPE : 'kagg:dataset', model.PROV_LOCATION : 'https://www.kaggle.com/mrisdal/fake-news',
                                        'kagg:title' : 'Getting Real about Fake News', 'kagg:resume' : 'Text & metadata from fake & biased news sources around the web',
                                        'kagg:usability' : '7.1', 'kagg:license' : 'CC0: Public Domain', 'kagg:tags' : 'arts and entertainment, news, politics, languages',
                                        'kagg:records' : '12.999', 'kagg:columns' : '20'})
zip = doc.entity('kagg:FakeNewData.zip', {model.PROV_TYPE : 'File', 'kagg:size' : '19.4 MB'})
csv = doc.entity('kagg:fake.csv', {model.PROV_TYPE : 'File', 'kagg:size' : '54.05 MB'})
clean_data = doc.entity('ppgi:Dataset com Dados Tratados')
proc_data = doc.entity('ppgi:Dataset com Texto Processado')
artigo = doc.entity('ppgi:Artigo sobre Análise Exploratória de Dados - Fake News')
# Criando agentes
meg = doc.agent('kagg:Meg Risdal', {model.PROV_TYPE : 'prov:Person', 'kagg:position': 'Product Manager', model.PROV_LOCATION : 'Los Angeles, California, United States',
                                    'kagg:user' : 'mrisdal', 'kagg:profile' : 'https://www.kaggle.com/mrisdal'})
bsdet = doc.agent('B.S. Detector', {model.PROV_TYPE : 'prov:SoftwareAgent', 'prov:publisher' : 'The Self Agency, LLC', 'prov:creator' : 'Daniel Sieradski',
                                    model.PROV_LOCATION : 'https://addons.mozilla.org/en-US/firefox/addon/bsdetector/'})
api = doc.agent('webhose.io API', {model.PROV_TYPE : 'prov:SoftwareAgent', 'prov:publisher' : 'Webhose', model.PROV_LOCATION : 'https://webhose.io/'})
sirius = doc.agent('ppgi:Sírius Thadeu', {model.PROV_TYPE : 'prov:Person', model.PROV_ROLE : 'Aluno', 'prov:organization' : 'PPGI-UFRJ',
                                          model.PROV_LOCATION : 'Rio de Janeiro - RJ'})
# Criando atividades
scrap = doc.activity('kagg:Scrap Text and Metadata', '25/10/2016', '25/11/2016', {'kagg:source' : '244 websites'})
data_clean = doc.activity('ppgi:Limpeza de Dados')
text_proc = doc.activity('ppgi:Processamento de Texto')
eda = doc.activity('ppgi:Análise Exploratória de Dados')
# Relacionamentos
doc.used(data_clean, dataset)
doc.used(text_proc, clean_data)
doc.used(eda, proc_data)
doc.wasInfluencedBy(scrap, api)
doc.wasInfluencedBy(scrap, bsdet)
scrap.wasAssociatedWith(meg)
data_clean.wasAssociatedWith(sirius)
text_proc.wasAssociatedWith(sirius)
eda.wasAssociatedWith(sirius)
dataset.wasAttributedTo(meg)
dataset.wasGeneratedBy(scrap)
zip.wasDerivedFrom(dataset)
csv.wasDerivedFrom(zip)
clean_data.wasGeneratedBy(data_clean)
clean_data.wasDerivedFrom(csv)
proc_data.wasGeneratedBy(text_proc)
proc_data.wasDerivedFrom(clean_data)
artigo.wasGeneratedBy(eda)
artigo.wasDerivedFrom(proc_data)
artigo.wasAttributedTo(sirius)
print(doc.get_provn())

## Gera o gráfico

In [ ]:
from prov import dot
# Save the provenance graph
dot = dot.prov_to_dot(doc)
dot.write_png('fake-news-dataset-prov.png')
# Visualize the graph file
from IPython.display import Image
Image('fake-news-dataset-prov.png')